# *오차역전파법*

## 4. 단순한 계층 구현하기

- 계산 그래프의 곱셈 노드를 'MulLayer', 덧셈 노드를 'AddLayer'로 구현
- 모든 계층은 forward()라는 순전파 메서드와 backward()라는 역전파 메서드로 구현
___

### 1) 곱셈 계층

In [1]:
 class MulLayer:
        def __init__(self):
            self.x = None
            self.y = None
            
            
        def forward(self, x, y):
            self.x = x
            self.y = y
            out = x * y
            
            return out
            
            
        def backward(self, dout):
            dx = dout * self.y
            dy = dout * self.x
            
            return dx, dy

- \__init__을 통해 인스턴스 변수 x와 y 초기화
- forward() 에서는 x와 y를 인수로 받아 곱셈값을 반환
- backward() 에서는 상류에서 넘어온 미분 값에 순전파 때의 값을 바꿔 곱함
- 이하 다음과 같은 사과 예제 구현
![](image/fig 5-16.png)

### - 순전파 프로세스

In [9]:
apple = 100
apple_num = 2
vat = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

apple_price = mul_apple_layer.forward(apple, apple_num)
the_price = mul_tax_layer.forward(apple_price, vat)

the_price


220.00000000000003

- MulLayer()로 각 노드 구성
- 각 노드에서 노드명(클래스).forward() 메서드를 통해 인수를 곱하여 계산 값 산출
___

### - 역전파 프로세스

In [12]:
dprice = 1

dapple_price, dvat = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dvat)

2.2 110.00000000000001 200


- 상류층의 미분값을 입력값으로 받아 계산
- 각 노드에서 노드명(클래스).backward()메서드를 통해 '상류층의 미분값'과 '순전파 때 보관한 x,y값'을 곱하여 선행노드의 미분값 계산
___

### 2) 덧셈 계층

In [13]:
class AddLayer():
    def __init__(self):
        pass
    
    
    def forward(self, x, y):
        return x + y
    
    
    def backward(self, dout):
        return dout, dout

- 덧셈 노드에서는 초기화가 불필요하니 #__init__()에서 아무 일도 하지 않음
- 순전파에서 따로 계산 값 및 변수 보관이 불필요하니 값 저장 지향
- 역전파에서도 단순히 상류의 미분값 전달
___
> ### 덧셈 계층과 곱셈 계층을 사용하여 사과 귤 구매 상황을 구현
![](image/fig 5-17.png)

In [14]:
a_count = 2
a_price = 100
o_count = 3
o_price = 150
vat = 1.1

In [15]:
ap_nod = MulLayer()
op_nod = MulLayer()
fp_nod = AddLayer()
tp_nod = MulLayer()

In [17]:
ap = ap_nod.forward(a_count, a_price)
op = op_nod.forward(o_count, o_price)
fp = fp_nod.forward(ap, op)
tp = tp_nod.forward(fp, vat)
tp

715.0000000000001

In [18]:
init_diff = 1
dfp, dvat = tp_nod.backward(init_diff)
dap, dop = fp_nod.backward(dfp)
da_count, da_price = ap_nod.backward(dap)
do_count, do_price = op_nod.backward(dop)

{'apple': [da_count, da_price], 'orange': [do_count, do_price], 'dvat': dvat}

{'apple': [110.00000000000001, 2.2],
 'dvat': 650,
 'orange': [165.0, 3.3000000000000003]}

- 필요한 계층 생성 (MulLayer, AddLayer)
- 순전파 메서드로 계산
- 역전파 메서드로 계산
___